In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy rustworkx scipy

# Algorithme d'optimisation approximative quantique

*Estimation d'utilisation : 22 minutes sur un processeur Heron r3 (REMARQUE : il s'agit uniquement d'une estimation. Votre temps d'exécution peut varier.)*
## Contexte
Ce tutoriel montre comment implémenter l'**algorithme d'optimisation approximative quantique (QAOA)** -- une méthode itérative hybride (quantique-classique) -- dans le cadre des patterns Qiskit. Vous résoudrez d'abord le problème de **coupe maximale** (ou **Max-Cut**) pour un petit graphe, puis vous apprendrez à l'exécuter à l'échelle utilitaire. Toutes les exécutions matérielles de ce tutoriel devraient s'effectuer dans la limite de temps du plan Open, accessible gratuitement.

Le problème Max-Cut est un problème d'optimisation difficile à résoudre (plus précisément, il s'agit d'un problème *NP-difficile*) avec de nombreuses applications en clustering, en science des réseaux et en physique statistique. Ce tutoriel considère un graphe de nœuds connectés par des arêtes, et vise à partitionner les nœuds en deux ensembles de manière à maximiser le nombre d'arêtes traversées par cette coupe.

![Illustration d'un problème de coupe maximale](../docs/images/tutorials/quantum-approximate-optimization-algorithm/maxcut-illustration.avif)
## Prérequis
Avant de commencer ce tutoriel, assurez-vous d'avoir installé les éléments suivants :
- Qiskit SDK v1.0 ou version ultérieure, avec le support de [visualisation](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.22 ou version ultérieure (`pip install qiskit-ibm-runtime`)

De plus, vous aurez besoin d'un accès à une instance sur [IBM Quantum Platform](/guides/cloud-setup). Notez que ce tutoriel ne peut pas être exécuté avec le plan Open, car il exécute des charges de travail utilisant des [sessions](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/session), qui ne sont disponibles qu'avec un accès au plan Premium.
## Configuration

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
from scipy.optimize import minimize
from collections import defaultdict
from typing import Sequence


from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

## Partie I. QAOA à petite échelle
La première partie de ce tutoriel utilise un problème Max-Cut à petite échelle pour illustrer les étapes de résolution d'un problème d'optimisation à l'aide d'un ordinateur quantique.

Pour donner un contexte avant de transposer ce problème en un algorithme quantique, vous pouvez mieux comprendre comment le problème Max-Cut devient un problème d'optimisation combinatoire classique en considérant d'abord la minimisation d'une fonction $f(x)$

$$
\min_{x\in {0, 1}^n}f(x),
$$

où l'entrée $x$ est un vecteur dont les composantes correspondent à chaque nœud d'un graphe. Ensuite, contraignez chacune de ces composantes à être soit $0$ soit $1$ (ce qui représente l'inclusion ou non dans la coupe). Cet exemple à petite échelle utilise un graphe avec $n=5$ nœuds.

Vous pourriez écrire une fonction d'une paire de nœuds $i,j$ qui indique si l'arête correspondante $(i,j)$ est dans la coupe. Par exemple, la fonction $x_i + x_j - 2 x_i x_j$ vaut 1 uniquement si l'un des deux $x_i$ ou $x_j$ vaut 1 (ce qui signifie que l'arête est dans la coupe) et zéro sinon. Le problème de maximisation des arêtes dans la coupe peut être formulé comme

$$
\max_{x\in {0, 1}^n} \sum_{(i,j)} x_i + x_j - 2 x_i x_j,
$$

ce qui peut être réécrit sous forme de minimisation

$$
\min_{x\in {0, 1}^n} \sum_{(i,j)}  2 x_i x_j - x_i - x_j.
$$

Le minimum de $f(x)$ dans ce cas correspond au moment où le nombre d'arêtes traversées par la coupe est maximal. Comme vous pouvez le constater, il n'y a encore rien qui relève de l'informatique quantique. Vous devez reformuler ce problème en quelque chose qu'un ordinateur quantique peut comprendre.
Initialisez votre problème en créant un graphe avec $n=5$ nœuds.

In [2]:
n = 5

graph = rx.PyGraph()
graph.add_nodes_from(np.arange(0, n, 1))
edge_list = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (0, 4, 1.0),
    (1, 2, 1.0),
    (2, 3, 1.0),
    (3, 4, 1.0),
]
graph.add_edges_from(edge_list)
draw_graph(graph, node_size=600, with_labels=True)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif" alt="Output of the previous code cell" />

![Sortie de la cellule de code précédente](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/6ced6bea-0.avif)

### Étape 1 : Transposer les entrées classiques en un problème quantique
La première étape du pattern consiste à transposer le problème classique (graphe) en **circuits** et **opérateurs** quantiques. Pour cela, trois étapes principales sont nécessaires :

1. Utiliser une série de reformulations mathématiques pour représenter ce problème en utilisant la notation des problèmes d'optimisation binaire quadratique sans contraintes (QUBO).
2. Réécrire le problème d'optimisation sous forme d'un hamiltonien dont l'état fondamental correspond à la solution qui minimise la fonction de coût.
3. Créer un circuit quantique qui préparera l'état fondamental de cet hamiltonien via un processus similaire au recuit quantique.

**Remarque :** Dans la méthodologie QAOA, vous souhaitez finalement obtenir un opérateur (**hamiltonien**) qui représente la **fonction de coût** de notre algorithme hybride, ainsi qu'un circuit paramétré (**Ansatz**) qui représente des états quantiques avec des solutions candidates au problème. Vous pouvez échantillonner à partir de ces états candidats puis les évaluer à l'aide de la fonction de coût.

#### Graphe &rarr; problème d'optimisation
La première étape de la transposition est un changement de notation. Ce qui suit exprime le problème en notation QUBO :

$$
\min_{x\in {0, 1}^n}x^T Q x,
$$

où $Q$ est une matrice $n\times n$ de nombres réels, $n$ correspond au nombre de nœuds dans votre graphe, $x$ est le vecteur de variables binaires introduit ci-dessus, et $x^T$ indique la transposée du vecteur $x$.

```
Maximize
 -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_4 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_3*x_4 + 3*x_0
 + 2*x_1 + 3*x_2 + 2*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4
```
### Problème d'optimisation &rarr; hamiltonien
Vous pouvez ensuite reformuler le problème QUBO sous forme d'un **hamiltonien** (ici, une matrice qui représente l'énergie d'un système) :

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

<details>
<summary>
**Étapes de reformulation du problème QAOA vers l'hamiltonien**
</summary>

Pour montrer comment le problème QAOA peut être réécrit de cette manière, remplacez d'abord les variables binaires $x_i$ par un nouvel ensemble de variables $z_i\in{-1, 1}$ via

$$
x_i = \frac{1-z_i}{2}.
$$

Ici, vous pouvez voir que si $x_i$ vaut $0$, alors $z_i$ doit valoir $1$. Lorsque les $x_i$ sont substitués par les $z_i$ dans le problème d'optimisation ($x^TQx$), une formulation équivalente peut être obtenue.

$$
x^TQx=\sum_{ij}Q_{ij}x_ix_j \\ =\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j) \\=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
$$

Maintenant, si nous définissons $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$, supprimons le préfacteur et le terme constant $n^2$, nous obtenons les deux formulations équivalentes du même problème d'optimisation.

$$
\min_{x\in{0,1}^n} x^TQx\Longleftrightarrow \min_{z\in{-1,1}^n}z^TQz + b^Tz
$$

Ici, $b$ dépend de $Q$. Notez que pour obtenir $z^TQz + b^Tz$ nous avons supprimé le facteur de 1/4 et un décalage constant de $n^2$ qui ne jouent aucun rôle dans l'optimisation.

Maintenant, pour obtenir une formulation quantique du problème, promouvez les variables $z_i$ en une matrice de Pauli $Z$, telle qu'une matrice $2\times 2$ de la forme

$$
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
$$

Lorsque vous substituez ces matrices dans le problème d'optimisation ci-dessus, vous obtenez l'hamiltonien suivant

$$
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
$$

*Rappelez-vous également que les matrices $Z$ sont intégrées dans l'espace de calcul de l'ordinateur quantique, c'est-à-dire un espace de Hilbert de taille $2^n\times 2^n$. Par conséquent, vous devez comprendre des termes tels que $Z_iZ_j$ comme le produit tensoriel $Z_i\otimes Z_j$ intégré dans l'espace de Hilbert $2^n\times 2^n$. Par exemple, dans un problème avec cinq variables de décision, le terme $Z_1Z_3$ doit être compris comme $I\otimes Z_3\otimes I\otimes Z_1\otimes I$ où $I$ est la matrice identité $2\times 2$.*
</details>

Cet hamiltonien est appelé l'**hamiltonien de la fonction de coût**. Il a la propriété que son état fondamental correspond à la solution qui **minimise la fonction de coût $f(x)$**.
Par conséquent, pour résoudre votre problème d'optimisation, vous devez maintenant préparer l'état fondamental de $H_C$ (ou un état ayant un fort recouvrement avec celui-ci) sur l'ordinateur quantique. Ensuite, l'échantillonnage à partir de cet état donnera, avec une forte probabilité, la solution à $min~f(x)$.
Considérons maintenant l'hamiltonien $H_C$ pour le problème **Max-Cut**. Soit chaque sommet du graphe associé à un qubit dans l'état $|0\rangle$ ou $|1\rangle$, où la valeur indique l'ensemble auquel le sommet appartient. L'objectif du problème est de maximiser le nombre d'arêtes $(v_1, v_2)$ pour lesquelles $v_1 = |0\rangle$ et $v_2 = |1\rangle$, ou vice-versa. Si nous associons l'opérateur $Z$ à chaque qubit, où

$$
    Z|0\rangle = |0\rangle \qquad Z|1\rangle = -|1\rangle
$$

alors une arête $(v_1, v_2)$ appartient à la coupe si la valeur propre de $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = -1$ ; en d'autres termes, les qubits associés à $v_1$ et $v_2$ sont différents. De même, $(v_1, v_2)$ n'appartient pas à la coupe si la valeur propre de $(Z_1|v_1\rangle) \cdot (Z_2|v_2\rangle) = 1$. Notez que nous ne nous intéressons pas à l'état exact du qubit associé à chaque sommet, mais uniquement à savoir s'ils sont identiques ou non de part et d'autre d'une arête. Le problème Max-Cut nous demande de trouver une assignation des qubits sur les sommets telle que la valeur propre de l'hamiltonien suivant soit minimisée
$$
    H_C = \sum_{(i,j) \in e} Q_{ij} \cdot Z_i Z_j.
$$

En d'autres termes, $b_i = 0$ pour tout $i$ dans le problème Max-Cut. La valeur de $Q_{ij}$ indique le poids de l'arête. Dans ce tutoriel, nous considérons un graphe non pondéré, c'est-à-dire $Q_{ij} = 1.0$ pour tout $i, j$.

In [ ]:
def build_max_cut_paulis(
    graph: rx.PyGraph,
) -> list[tuple[str, list[int], float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        weight = graph.get_edge_data(edge[0], edge[1])
        pauli_list.append(("ZZ", [edge[0], edge[1]], weight))
    return pauli_list


max_cut_paulis = build_max_cut_paulis(graph)
cost_hamiltonian = SparsePauliOp.from_sparse_list(max_cut_paulis, n)
print("Cost Function Hamiltonian:", cost_hamiltonian)

Cost Function Hamiltonian: SparsePauliOp(['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


#### Hamiltonian &rarr; quantum circuit

The Hamiltonian $H_C$ contains the quantum definition of your problem. Now you can create a quantum circuit that will help *sample* good solutions from the quantum computer. The QAOA is inspired by quantum annealing and applies alternating layers of operators in the quantum circuit.

The general idea is to start in the ground state of a known system, $H^{\otimes n}|0\rangle$ above, and then steer the system into the ground state of the cost operator that you are interested in. This is done by applying the operators $\exp\{-i\gamma_k H_C\}$ and $\exp\{-i\beta_k H_m\}$ with angles $\gamma_1,...,\gamma_p$ and $\beta_1,...,\beta_p~$.


The quantum circuit that you generate is **parametrized** by $\gamma_i$ and $\beta_i$, so you can try out different values of $\gamma_i$ and $\beta_i$ and sample from the resulting state.

![Circuit diagram with QAOA layers](../docs/images/tutorials/quantum-approximate-optimization-algorithm/circuit-diagram.svg)


In this case, you will try an example with one QAOA layer that contains two parameters: $\gamma_1$ and $\beta_1$.

In [4]:
circuit = QAOAAnsatz(cost_operator=cost_hamiltonian, reps=2)
circuit.measure_all()

circuit.draw("mpl")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/7bd8c6d4-f40f-4a11-a440-0b26d9021b53-0.avif" alt="Output of the previous code cell" />

In [5]:
circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

### Step 2: Optimize problem for quantum hardware execution

The circuit above contains a series of abstractions useful to think about quantum algorithms, but not possible to run on the hardware. To be able to run on a QPU, the circuit needs to undergo a series of operations that make up the **transpilation** or **circuit optimization** step of the pattern.

The Qiskit library offers a series of **transpilation passes** that cater to a wide range of circuit transformations. You need to make sure that your circuit is **optimized** for your purpose.

Transpilation may involves several steps, such as:

* **Initial mapping** of the qubits in the circuit (such as decision variables) to physical qubits on the device.
* **Unrolling** of the instructions in the quantum circuit to the hardware-native instructions that the backend understands.
* **Routing** of any qubits in the circuit that interact to physical qubits that are adjacent with one another.
* **Error suppression** by adding single-qubit gates to suppress noise with dynamical decoupling.


More information about transpilation is available in our [documentation](/docs/guides/transpile).

The following code transforms and optimizes the abstract circuit into a format that is ready for execution on one of devices accessible through the cloud using the **Qiskit IBM Runtime service**.

In [6]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit = pm.run(circuit)
candidate_circuit.draw("mpl", fold=False, idle_wires=False)

<IBMBackend('test_heron_pok_1')>


<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives

In the QAOA workflow, the optimal QAOA parameters are found in an iterative optimization loop, which runs a series of circuit evaluations and uses a classical optimizer to find the optimal $\beta_k$ and $\gamma_k$ parameters. This execution loop is executed via the following steps:

1. Define the initial parameters
2. Instantiate a new `Session` containing the optimization loop and the primitive used to sample the circuit
3. Once an optimal set of parameters is found, execute the circuit a final time to obtain a final distribution which will be used in the post-process step.

#### Define circuit with initial parameters
We start with arbitrary chosen parameters.

In [7]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_beta, initial_gamma, initial_gamma]

### Étape 2 : Optimiser le problème pour l'exécution sur du matériel quantique
Le circuit ci-dessus contient une série d'abstractions utiles pour raisonner sur les algorithmes quantiques, mais qui ne peuvent pas être exécutées directement sur le matériel. Pour pouvoir s'exécuter sur un QPU, le circuit doit subir une série d'opérations qui constituent l'étape de **transpilation** ou d'**optimisation de circuit** du pattern.

La bibliothèque Qiskit offre une série de **passes de transpilation** qui couvrent un large éventail de transformations de circuits. Vous devez vous assurer que votre circuit est **optimisé** pour votre objectif.

La transpilation peut impliquer plusieurs étapes, telles que :

* **Mappage initial** des qubits dans le circuit (tels que les variables de décision) vers les qubits physiques du dispositif.
* **Décomposition** des instructions du circuit quantique en instructions natives du matériel que le Backend comprend.
* **Routage** de tout qubit du circuit qui interagit vers des qubits physiques adjacents les uns aux autres.
* **Suppression d'erreurs** par l'ajout de portes à qubit unique pour supprimer le bruit avec le découplage dynamique.

Plus d'informations sur la transpilation sont disponibles dans notre [documentation](/guides/transpile).

Le code suivant transforme et optimise le circuit abstrait dans un format prêt à être exécuté sur l'un des dispositifs accessibles via le cloud en utilisant le **service Qiskit IBM Runtime**.

In [8]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):
    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs

    objective_func_vals.append(cost)

    return cost

In [9]:
objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, cost_hamiltonian, estimator),
        method="COBYLA",
        tol=1e-2,
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -1.6295230263157894
       x: [ 1.530e+00  1.439e+00  4.071e+00  4.434e+00]
    nfev: 26
   maxcv: 0.0


![Sortie de la cellule de code précédente](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3f28a422-805c-4d3d-b5f6-62539e9133bd-1.avif)

### Étape 3 : Exécuter à l'aide des primitives Qiskit
Dans le flux de travail QAOA, les paramètres QAOA optimaux sont trouvés dans une boucle d'optimisation itérative, qui exécute une série d'évaluations de circuits et utilise un optimiseur classique pour trouver les paramètres $\beta_k$ et $\gamma_k$ optimaux. Cette boucle d'exécution est réalisée via les étapes suivantes :

1. Définir les paramètres initiaux
2. Instancier une nouvelle `Session` contenant la boucle d'optimisation et la primitive utilisée pour échantillonner le circuit
3. Une fois qu'un ensemble optimal de paramètres est trouvé, exécuter le circuit une dernière fois pour obtenir une distribution finale qui sera utilisée dans l'étape de post-traitement.
#### Définir le circuit avec les paramètres initiaux
Nous commençons avec des paramètres choisis arbitrairement.

In [10]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif" alt="Output of the previous code cell" />

#### Définir le Backend et la primitive d'exécution
Utilisez les **primitives Qiskit Runtime** pour interagir avec les Backends IBM&reg;. Les deux primitives sont Sampler et Estimator, et le choix de la primitive dépend du type de mesure que vous souhaitez effectuer sur l'ordinateur quantique. Pour la minimisation de $H_C$, utilisez l'Estimator puisque la mesure de la fonction de coût est simplement la valeur d'espérance de $\langle H_C \rangle$.
#### Exécution
Les primitives offrent une variété de [modes d'exécution](/guides/execution-modes) pour planifier les charges de travail sur les dispositifs quantiques, et un flux de travail QAOA s'exécute de manière itérative dans une session.

![Illustration montrant le comportement des modes d'exécution Job unique, Batch et Session.](../docs/images/tutorials/quantum-approximate-optimization-algorithm/runtime-modes.avif)

Vous pouvez intégrer la fonction de coût basée sur le Sampler dans la routine de minimisation SciPy pour trouver les paramètres optimaux.

In [11]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)
optimized_circuit.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/2989e76e-4296-4dd8-b065-2b8fced064cf-0.avif" alt="Output of the previous code cell" />

In [12]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub = (optimized_circuit,)
job = sampler.run([pub], shots=int(1e4))
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val / shots for key, val in counts_int.items()}
final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
print(final_distribution_int)

{28: 0.0328, 11: 0.0343, 2: 0.0296, 25: 0.0308, 16: 0.0303, 27: 0.0302, 13: 0.0323, 7: 0.0312, 4: 0.0296, 9: 0.0295, 26: 0.0321, 30: 0.031, 23: 0.0324, 31: 0.0303, 21: 0.0335, 15: 0.0317, 12: 0.0309, 29: 0.0297, 3: 0.0313, 5: 0.0312, 6: 0.0274, 10: 0.0329, 22: 0.0353, 0: 0.0315, 20: 0.0326, 8: 0.0322, 14: 0.0306, 17: 0.0295, 18: 0.0279, 1: 0.0325, 24: 0.0334, 19: 0.0295}


### Step 4: Post-process and return result in desired classical format

The post-processing step interprets the sampling output to return a solution for your original problem. In this case, you are interested in the bitstring with the highest probability as this determines the optimal cut. The symmetries in the problem allow for four possible solutions, and the sampling process will return one of them with a slightly higher probability, but you can see in the plotted distribution below that four of the bitstrings are distinctively more likely than the rest.

In [13]:
# auxiliary functions to sample most likely bitstring
def to_bitstring(integer, num_bits):
    result = np.binary_repr(integer, width=num_bits)
    return [int(digit) for digit in result]


keys = list(final_distribution_int.keys())
values = list(final_distribution_int.values())
most_likely = keys[np.argmax(np.abs(values))]
most_likely_bitstring = to_bitstring(most_likely, len(graph))
most_likely_bitstring.reverse()

print("Result bitstring:", most_likely_bitstring)

Result bitstring: [0, 1, 1, 0, 1]


In [14]:
matplotlib.rcParams.update({"font.size": 10})
final_bits = final_distribution_bin
values = np.abs(list(final_bits.values()))
top_4_values = sorted(values, reverse=True)[:4]
positions = []
for value in top_4_values:
    positions.append(np.where(values == value)[0])
fig = plt.figure(figsize=(11, 6))
ax = fig.add_subplot(1, 1, 1)
plt.xticks(rotation=45)
plt.title("Result Distribution")
plt.xlabel("Bitstrings (reversed)")
plt.ylabel("Probability")
ax.bar(list(final_bits.keys()), list(final_bits.values()), color="tab:grey")
for p in positions:
    ax.get_children()[int(p[0])].set_color("tab:purple")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif" alt="Output of the previous code cell" />

![Sortie de la cellule de code précédente](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/e14ecc92-0.avif)

Une fois que vous avez trouvé les paramètres optimaux pour le circuit, vous pouvez assigner ces paramètres et échantillonner la distribution finale obtenue avec les paramètres optimisés. C'est ici que la primitive *Sampler* doit être utilisée puisque c'est la distribution de probabilité des mesures de chaînes de bits qui correspond à la coupe optimale du graphe.

**Remarque :** Cela signifie préparer un état quantique $\psi$ dans l'ordinateur puis le mesurer. Une mesure fera s'effondrer l'état en un seul état de base computationnelle -- par exemple, `010101110000...` -- qui correspond à une solution candidate $x$ à notre problème d'optimisation initial ($\max f(x)$ ou $\min f(x)$ selon la tâche).

In [15]:
# auxiliary function to plot graphs
def plot_result(G, x):
    colors = ["tab:grey" if i == 0 else "tab:purple" for i in x]
    pos, _default_axes = rx.spring_layout(G), plt.axes(frameon=True)
    rx.visualization.mpl_draw(
        G, node_color=colors, node_size=100, alpha=0.8, pos=pos
    )


plot_result(graph, most_likely_bitstring)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif" alt="Output of the previous code cell" />

And calculate the value of the cut:

In [16]:
def evaluate_sample(x: Sequence[int], graph: rx.PyGraph) -> float:
    assert len(x) == len(
        list(graph.nodes())
    ), "The length of x must coincide with the number of nodes in the graph."
    return sum(
        x[u] * (1 - x[v]) + x[v] * (1 - x[u])
        for u, v in list(graph.edge_list())
    )


cut_value = evaluate_sample(most_likely_bitstring, graph)
print("The value of the cut is:", cut_value)

The value of the cut is: 5


## Part II. Scale it up!

You have access to many devices with over 100 qubits on IBM Quantum&reg; Platform. Select one on which to solve Max-Cut on a 100-node weighted graph. This is a "utility-scale" problem. The steps to build the workflow are followed the same as above, but with a much larger graph.

In [17]:
n = 100  # Number of nodes in graph
graph_100 = rx.PyGraph()
graph_100.add_nodes_from(np.arange(0, n, 1))
elist = []
for edge in backend.coupling_map:
    if edge[0] < n and edge[1] < n:
        elist.append((edge[0], edge[1], 1.0))
graph_100.add_edges_from(elist)
draw_graph(graph_100, node_size=200, with_labels=True, width=1)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif" alt="Output of the previous code cell" />

### Étape 4 : Post-traiter et retourner le résultat dans le format classique souhaité
L'étape de post-traitement interprète la sortie de l'échantillonnage pour retourner une solution à votre problème original. Dans ce cas, vous vous intéressez à la chaîne de bits avec la plus haute probabilité car elle détermine la coupe optimale. Les symétries du problème permettent quatre solutions possibles, et le processus d'échantillonnage en retournera une avec une probabilité légèrement plus élevée, mais vous pouvez voir dans la distribution tracée ci-dessous que quatre des chaînes de bits sont nettement plus probables que les autres.

In [18]:
max_cut_paulis_100 = build_max_cut_paulis(graph_100)

cost_hamiltonian_100 = SparsePauliOp.from_sparse_list(max_cut_paulis_100, 100)
print("Cost Function Hamiltonian:", cost_hamiltonian_100)

Cost Function Hamiltonian: SparsePauliOp(['...'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0

#### Hamiltonian &rarr; quantum circuit

In [19]:
circuit_100 = QAOAAnsatz(cost_operator=cost_hamiltonian_100, reps=1)
circuit_100.measure_all()

circuit_100.draw("mpl", fold=False, scale=0.2, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum execution
To scale the circuit optimization step to utility-scale problems, you can take advantage of the high performance transpilation strategies introduced in Qiskit SDK v1.0. Other tools include the new transpiler service with [AI enhanced transpiler passes](/docs/guides/ai-transpiler-passes).

In [20]:
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

candidate_circuit_100 = pm.run(circuit_100)
candidate_circuit_100.draw("mpl", fold=False, scale=0.1, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/3a14e7ad-0.avif" alt="Output of the previous code cell" />

![Sortie de la cellule de code précédente](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/650875e9-adbc-43bd-9505-556be2566278-0.avif)

#### Visualiser la meilleure coupe
À partir de la chaîne de bits optimale, vous pouvez ensuite visualiser cette coupe sur le graphe original.

In [21]:
initial_gamma = np.pi
initial_beta = np.pi / 2
init_params = [initial_beta, initial_gamma]

objective_func_vals = []  # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)

    estimator.options.default_shots = 1000

    # Set simple error suppression/mitigation options
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit_100, cost_hamiltonian_100, estimator),
        method="COBYLA",
    )
    print(result)

 message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: 0
     fun: -3.9939191365979383
       x: [ 1.571e+00  3.142e+00]
    nfev: 29
   maxcv: 0.0


![Sortie de la cellule de code précédente](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/33135970-8bc4-4fb2-ab87-08726a432ce4-0.avif)

Et calculer la valeur de la coupe :

In [22]:
optimized_circuit_100 = candidate_circuit_100.assign_parameters(result.x)
optimized_circuit_100.draw("mpl", fold=False, idle_wires=False)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/1c432c2e-0.avif" alt="Output of the previous code cell" />

Finally, execute the circuit with the optimal parameters to sample from the corresponding distribution.

In [23]:
# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"


pub = (optimized_circuit_100,)
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {
    key: val / shots for key, val in counts_int.items()
}

## Partie II. Passage à l'échelle !
Vous avez accès à de nombreux dispositifs de plus de 100 qubits sur IBM Quantum&reg; Platform. Sélectionnez-en un sur lequel résoudre le problème Max-Cut pour un graphe pondéré de 100 nœuds. Il s'agit d'un problème à « échelle utilitaire ». Les étapes pour construire le flux de travail sont les mêmes que ci-dessus, mais avec un graphe beaucoup plus grand.

In [24]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/0fda3611-0.avif" alt="Output of the previous code cell" />

![Sortie de la cellule de code précédente](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/590fe2ce-0.avif)

### Étape 1 : Transposer les entrées classiques en un problème quantique
#### Graphe &rarr; hamiltonien
Commencez par convertir le graphe que vous souhaitez résoudre directement en un hamiltonien adapté au QAOA.

In [25]:
_PARITY = np.array(
    [-1 if bin(i).count("1") % 2 else 1 for i in range(256)],
    dtype=np.complex128,
)


def evaluate_sparse_pauli(state: int, observable: SparsePauliOp) -> complex:
    """Utility for the evaluation of the expectation value of a measured state."""
    packed_uint8 = np.packbits(observable.paulis.z, axis=1, bitorder="little")
    state_bytes = np.frombuffer(
        state.to_bytes(packed_uint8.shape[1], "little"), dtype=np.uint8
    )
    reduced = np.bitwise_xor.reduce(packed_uint8 & state_bytes, axis=1)
    return np.sum(observable.coeffs * _PARITY[reduced])


def best_solution(samples, hamiltonian):
    """Find solution with lowest cost"""
    min_cost = 1000
    min_sol = None
    for bit_str in samples.keys():
        # Qiskit use little endian hence the [::-1]
        candidate_sol = int(bit_str)
        # fval = qp.objective.evaluate(candidate_sol)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        if fval <= min_cost:
            min_sol = candidate_sol

    return min_sol


best_sol_100 = best_solution(final_distribution_100_int, cost_hamiltonian_100)
best_sol_bitstring_100 = to_bitstring(int(best_sol_100), len(graph_100))
best_sol_bitstring_100.reverse()

print("Result bitstring:", best_sol_bitstring_100)

Result bitstring: [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]


Next, visualize the cut. Nodes of the same color belong to the same group.

In [26]:
plot_result(graph_100, best_sol_bitstring_100)

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/b4a25e28-0.avif" alt="Output of the previous code cell" />

#### Hamiltonien &rarr; circuit quantique

In [27]:
cut_value_100 = evaluate_sample(best_sol_bitstring_100, graph_100)
print("The value of the cut is:", cut_value_100)

The value of the cut is: 124


![Sortie de la cellule de code précédente](../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/9693adfc-0.avif)

### Étape 2 : Optimiser le problème pour l'exécution quantique
Pour passer à l'échelle l'étape d'optimisation de circuit aux problèmes à échelle utilitaire, vous pouvez tirer parti des stratégies de transpilation haute performance introduites dans le Qiskit SDK v1.0. D'autres outils incluent le nouveau service de transpilation avec des [passes de transpilation améliorées par l'IA](/guides/ai-transpiler-passes).

In [28]:
# auxiliary function to help plot cumulative distribution functions
def _plot_cdf(objective_values: dict, ax, color):
    x_vals = sorted(objective_values.keys(), reverse=True)
    y_vals = np.cumsum([objective_values[x] for x in x_vals])
    ax.plot(x_vals, y_vals, color=color)


def plot_cdf(dist, ax, title):
    _plot_cdf(
        dist,
        ax,
        "C1",
    )
    ax.vlines(min(list(dist.keys())), 0, 1, "C1", linestyle="--")

    ax.set_title(title)
    ax.set_xlabel("Objective function value")
    ax.set_ylabel("Cumulative distribution function")
    ax.grid(alpha=0.3)


# auxiliary function to convert bit-strings to objective values
def samples_to_objective_values(samples, hamiltonian):
    """Convert the samples to values of the objective function."""

    objective_values = defaultdict(float)
    for bit_str, prob in samples.items():
        candidate_sol = int(bit_str)
        fval = evaluate_sparse_pauli(candidate_sol, hamiltonian).real
        objective_values[fval] += prob

    return objective_values

In [29]:
result_dist = samples_to_objective_values(
    final_distribution_100_int, cost_hamiltonian_100
)

Finally, you can plot the cumulative distribution function to visualize how each sample contributes to the total probability distribution and the corresponding objective value. The horizontal spread shows the range of objective values of the samples in the final distribution. Ideally, you would see that the cumulative distribution function has "jumps" at the lower end of the objective function value axis. This would mean that few solutions with low cost have high probability of being sampled. A smooth, wide curve indicates that each sample is similarly likely, and they can have very different objective values, low or high.

In [30]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_cdf(result_dist, ax, "Eagle device")

<Image src="../docs/images/tutorials/quantum-approximate-optimization-algorithm/extracted-outputs/4381a2b3-0.avif" alt="Output of the previous code cell" />

Une fois les paramètres optimaux trouvés en exécutant le QAOA sur le dispositif, assignez les paramètres au circuit.